In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')



# terus ini nama file yang kita coba buat latihan
df=pd.read_csv('/content/gdrive/My Drive/SNI/TULT1 - 15-10-2023.csv')

df = df.loc[:,['created_at','entry_id','field1','field2','field3','field4','field5']]

# HILANGIN TULISAN T DAN Z YANG ADA
df['created_at']=df['created_at'].str.replace("T"," ")
df['created_at']=df['created_at'].str.replace("Z"," ")

df.rename(columns={'field1': 'Tekanan Udara', 'field2': 'Temperatur','field3': 'Kelembapan Relatif', 'field4' : 'PM2.5', 'field5' : 'CO2'}, inplace=True)
df = df.dropna()
#change to local time +7
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_convert('Asia/Jakarta')


# hilangkan nilai co2 dan pm2.5 yang bernilai <= 0
df=df[(df['CO2']>0)]
df=df[(df['PM2.5']>0)]

#
df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-3-fd841b056633>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_convert('Asia/Jakarta')


,created_at,entry_id,Tekanan Udara,Temperatur,Kelembapan Relatif,PM2.5,CO2
6047,2023-03-29 21:33:03+07:00,6048,75841.72656,26.9,63.8,32.0,3011.36011
6048,2023-03-29 21:35:06+07:00,6049,75841.72656,26.5,64.3,31.0,3209.91992
6049,2023-03-29 21:37:08+07:00,6050,75841.72656,26.5,64.3,31.0,3209.91992
6050,2023-03-29 21:39:11+07:00,6051,75841.72656,26.5,64.3,29.0,2797.53003
6051,2023-03-29 21:41:13+07:00,6052,75841.72656,26.5,64.3,29.0,2797.53003
...,...,...,...,...,...,...,...
128194,2023-10-09 11:23:28+07:00,128195,93093.72656,33.2,41.5,12.0,567.57001
128195,2023-10-09 11:24:58+07:00,128196,93088.78125,33.0,41.8,9.0,399.56000
128196,2023-10-09 11:26:28+07:00,128197,93090.57813,32.8,42.2,9.0,399.56000
128630,2023-10-11 10:34:49+07:00,128631,-4249.87012,32.8,49.1,20.0,1086.88000


In [ ]:
# Ambil Kelembapan Relatif & PM 2.5 (KITA TAMBAHKAN INDEX BULAN, HARI DAN JAM) UNTUK DISORTING NANTI
df1 = df.loc[:,['created_at','entry_id','Temperatur','Kelembapan Relatif','PM2.5','CO2']]
df1['Bulan'] = pd.DatetimeIndex(df['created_at']).month
df1.set_index = df1['Bulan'].inplace=True
df1['Hari'] = pd.DatetimeIndex(df['created_at']).day
df1.set_index = df1['Hari'].inplace=True
df1['Jam'] = pd.DatetimeIndex(df['created_at']).hour
df1.set_index = df1['Jam'].inplace=True

df1

,created_at,entry_id,Temperatur,Kelembapan Relatif,PM2.5,CO2,Bulan,Hari,Jam
6047,2023-03-29 21:33:03+07:00,6048,26.9,63.8,32.0,3011.36011,3,29,21
6048,2023-03-29 21:35:06+07:00,6049,26.5,64.3,31.0,3209.91992,3,29,21
6049,2023-03-29 21:37:08+07:00,6050,26.5,64.3,31.0,3209.91992,3,29,21
6050,2023-03-29 21:39:11+07:00,6051,26.5,64.3,29.0,2797.53003,3,29,21
6051,2023-03-29 21:41:13+07:00,6052,26.5,64.3,29.0,2797.53003,3,29,21
...,...,...,...,...,...,...,...,...,...
128194,2023-10-09 11:23:28+07:00,128195,33.2,41.5,12.0,567.57001,10,9,11
128195,2023-10-09 11:24:58+07:00,128196,33.0,41.8,9.0,399.56000,10,9,11
128196,2023-10-09 11:26:28+07:00,128197,32.8,42.2,9.0,399.56000,10,9,11
128630,2023-10-11 10:34:49+07:00,128631,32.8,49.1,20.0,1086.88000,10,11,10


In [ ]:
#Validasi CO2


# terus kita filter data nya per 2 menit(sesuai ama waktu register data thingspeak)
dfco2 = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()

#Hapus nilai CO2 yang di luar batas range
dfco2 = dfco2[(dfco2['CO2']>400) &  (dfco2['CO2'] <5000)]

# kita ambil nilai index nya sesuai yang ada di petik '',
dfco2 = dfco2.loc[:,['created_at','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari','Jam']]

# dropna() fungsinya buat drop data yang kosong
dfco2 = dfco2.dropna()

# astype int ngubah angka ke int
dfco2['CO2']=dfco2['CO2'].round().astype(int)
dfco2['Bulan']=dfco2['Bulan'].round().astype(int)
dfco2['Hari']=dfco2['Hari'].round().astype(int)
dfco2['Jam']=dfco2['Jam'].round().astype(int)

#round(2) gunanya buat ngatur 2 angka dibelakang koma, bisa atur angka apa aja
dfco2

,created_at,Temperatur,Kelembapan Relatif,CO2,Bulan,Hari,Jam
0,2023-03-29 21:32:00+07:00,26.90,63.80,3011,3,29,21
1,2023-03-29 21:34:00+07:00,26.50,64.30,3210,3,29,21
2,2023-03-29 21:36:00+07:00,26.50,64.30,3210,3,29,21
3,2023-03-29 21:38:00+07:00,26.50,64.30,2798,3,29,21
4,2023-03-29 21:40:00+07:00,26.50,64.30,2798,3,29,21
...,...,...,...,...,...,...,...
139372,2023-10-09 11:16:00+07:00,33.40,46.10,583,10,9,11
139374,2023-10-09 11:20:00+07:00,33.45,41.75,445,10,9,11
139375,2023-10-09 11:22:00+07:00,33.20,41.50,568,10,9,11
140791,2023-10-11 10:34:00+07:00,32.80,49.10,1087,10,11,10


In [ ]:
#Outlier windowing per jam CO2
dfoutlier = pd.DataFrame(
    {
        "created_at": ["1990-10-09 00:00:00"],
        "Temperatur": ["0"],
        "Kelembapan Relatif": ["0"],
        "CO2": ["0"],
    },
    index=[0, 1, 2, 3],
)

x=1
while x<=10 :
  y=1
  while y<=31 :
    z=0
    while z<=23 :
      dfvalidasi = dfco2[(dfco2['Bulan']==x) & (dfco2['Hari']==y) & (dfco2['Jam']==z)]
      dfvalidasi = dfvalidasi.loc[:,['created_at','Temperatur','Kelembapan Relatif','CO2']]
      data = len(dfvalidasi.axes[0])
      jumlah = str(data)
      if (dfvalidasi.empty==True):
        pass
      else :
        Q3 = dfvalidasi['CO2'].describe()['75%']
        Q1 = dfvalidasi['CO2'].describe()['25%']
        IQR = Q3 - Q1
        UF = Q3 + (1.5*IQR)
        LF = Q1 - (1.5*IQR)
        dfvalidasi = dfvalidasi[(dfvalidasi['CO2'] < UF) & (dfvalidasi['CO2'] > LF)]

        dfoutlier = pd.concat([dfoutlier, dfvalidasi], axis=0, ignore_index=True)
      z+=1
    y+=1
  x+=1

dfoutlier['CO2'] = dfoutlier['CO2'].astype(int)
dfoutlier = dfoutlier[(dfoutlier['CO2']>400) &  (dfoutlier['CO2'] <5000)]

dfoutlier

In [ ]:
dfcount = dfoutlier
dfstd = dfoutlier


dfcount = dfcount.groupby(pd.Grouper(key='created_at',freq='1H')).count()[lambda x: x > 22].dropna().round().astype(int)
# ambil 1 parameter hanya untuk menghitung ketersediaan data
dfcount = dfcount['CO2'].reset_index()
dfcount = dfcount.rename(columns={'CO2':'(n)Data'})

# hitung std CO2
dfstd = dfstd.groupby(pd.Grouper(key='created_at',freq='1H')).std()
dfstd = dfstd['CO2'].reset_index().round(2)
dfstd = dfstd.rename(columns={'CO2':'σ (ppm)'})

dfdata = pd.merge(dfcount, dfstd, on='created_at', how='inner')



# ambil 1 parameter

In [ ]:
# Gabungkan data per jam hanya dengan data yang tersedia 75%
df1hco2 = dfoutlier
df1hco2 = df1hco2.groupby(pd.Grouper(key='created_at',freq='1H')).mean().round(2)

df1hco2 = pd.merge(dfdata, df1hco2, on='created_at', how='inner')

df1hco2['CV (%)'] = ((df1hco2['σ (ppm)'] / df1hco2['CO2']) * 100).round(2)

df1hco2 = df1hco2.loc[:,['created_at','(n)Data','σ (ppm)','CV (%)','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari']]


df1hco2

,created_at,(n)Data,σ (ppm),CV (%),Temperatur,Kelembapan Relatif,CO2,Bulan,Hari
0,2023-04-06 17:00:00+07:00,30,78.87,8.65,27.47,47.90,911.70,4.0,6.0
1,2023-04-06 19:00:00+07:00,24,152.35,24.99,28.64,54.61,609.54,4.0,6.0
2,2023-04-06 20:00:00+07:00,29,48.72,8.45,29.07,57.16,576.45,4.0,6.0
3,2023-04-06 21:00:00+07:00,30,76.40,12.23,28.78,50.63,624.60,4.0,6.0
4,2023-04-06 22:00:00+07:00,29,66.40,10.03,28.52,51.63,661.76,4.0,6.0
...,...,...,...,...,...,...,...,...,...
1489,2023-10-09 06:00:00+07:00,30,94.45,10.94,24.01,79.93,863.40,10.0,9.0
1490,2023-10-09 07:00:00+07:00,30,115.47,14.74,26.23,70.98,783.43,10.0,9.0
1491,2023-10-09 08:00:00+07:00,30,93.25,13.26,28.44,62.69,703.30,10.0,9.0
1492,2023-10-09 09:00:00+07:00,30,130.47,19.38,29.42,53.70,673.27,10.0,9.0


In [ ]:
path = '/content/gdrive/My Drive/SNI/TULT1/CO2/Hari/'


x=1
while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    dfhari = df1hco2[(df1hco2['Bulan']==x) & (df1hco2['Hari']==y) ]

    data = len(dfhari.axes[0])
    jumlah = str(data)
    if (dfhari.empty==True):
      pass
    else :
      std = dfhari['CO2'].std(numeric_only=True).round(2)
      mean = dfhari['CO2'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      dfhari.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
# 1H in Months
path = '/content/gdrive/My Drive/SNI/TULT1/CO2/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df1hco2[(df1hco2['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['CO2'].std(numeric_only=True).round(2)
    mean = dfbulan['CO2'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation 1H.csv", index=None)
  x+=1

In [ ]:
df8hco2 = df1hco2
df8hco2 = df8hco2.groupby(pd.Grouper(key='created_at',freq='8H')).mean().round(2).reset_index()
df8hco2 = df8hco2.loc[:,['created_at','σ (ppm)','CV (%)','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari']]



In [ ]:
df8hco2

,created_at,σ (ppm),CV (%),Temperatur,Kelembapan Relatif,CO2,Bulan,Hari
0,2023-04-06 16:00:00+07:00,77.32,11.68,28.52,53.08,683.98,4.0,6.0
1,2023-04-07 00:00:00+07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-04-07 08:00:00+07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-04-07 16:00:00+07:00,63.21,10.13,28.51,64.99,624.00,4.0,7.0
4,2023-04-08 00:00:00+07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
553,2023-10-08 00:00:00+07:00,98.33,11.30,23.67,81.42,870.62,10.0,8.0
554,2023-10-08 08:00:00+07:00,92.77,13.89,30.43,56.00,666.12,10.0,8.0
555,2023-10-08 16:00:00+07:00,99.00,12.91,26.29,69.32,774.97,10.0,8.0
556,2023-10-09 00:00:00+07:00,101.38,11.55,23.29,83.80,882.85,10.0,9.0


In [ ]:
# 8H in Months
path = '/content/gdrive/My Drive/SNI/TULT1/CO2/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df8hco2[(df8hco2['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['CO2'].std(numeric_only=True).round(2)
    mean = dfbulan['CO2'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation 8H.csv", index=None)
  x+=1

Nah, disini tuh seharusnya kita validasi langsung aja per jam dulu cari outlier, terus kalau data per jamnya kurang dari 22.5 (75% perjam) di drop di jam itu, kalau udah, baru kita sorting.

# PM2.5

In [ ]:
#Validasi PM2.5


dfpm = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()

#Hapus nilai PM2.5 yang di luar batas range
dfpm = dfpm[(dfpm['PM2.5']>0) &  (dfpm['PM2.5'] <250)]

#Correction factor for RH >= 80
dfpm.loc[dfpm['Kelembapan Relatif']>=80, ['PM2.5']] *= 0.67

dfpm = dfpm.loc[:,['created_at','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari','Jam']]

dfpm = dfpm.dropna()

# astype int ngubah angka ke int
dfpm['PM2.5']=dfpm['PM2.5'].round().astype(int)
dfpm['Bulan']=dfpm['Bulan'].round().astype(int)
dfpm['Hari']=dfpm['Hari'].round().astype(int)
dfpm['Jam']=dfpm['Jam'].round().astype(int)

dfpm

,created_at,Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari,Jam
0,2023-03-29 21:32:00+07:00,26.9,63.8,32,3,29,21
1,2023-03-29 21:34:00+07:00,26.5,64.3,31,3,29,21
2,2023-03-29 21:36:00+07:00,26.5,64.3,31,3,29,21
3,2023-03-29 21:38:00+07:00,26.5,64.3,29,3,29,21
4,2023-03-29 21:40:00+07:00,26.5,64.3,29,3,29,21
...,...,...,...,...,...,...,...
139375,2023-10-09 11:22:00+07:00,33.2,41.5,12,10,9,11
139376,2023-10-09 11:24:00+07:00,33.0,41.8,9,10,9,11
139377,2023-10-09 11:26:00+07:00,32.8,42.2,9,10,9,11
140791,2023-10-11 10:34:00+07:00,32.8,49.1,20,10,11,10


In [ ]:
dfcount = dfpm
dfstd = dfpm


dfcount = dfcount.groupby(pd.Grouper(key='created_at',freq='1H')).count()[lambda x: x > 22].dropna().round().astype(int)
# ambil 1 parameter hanya untuk menghitung ketersediaan data
dfcount = dfcount['PM2.5'].reset_index()
dfcount = dfcount.rename(columns={'PM2.5':'(n)Data'})

# hitung std PM2.5
dfstd = dfstd.groupby(pd.Grouper(key='created_at',freq='1H')).std()
dfstd = dfstd['PM2.5'].reset_index().round(2)
dfstd = dfstd.rename(columns={'PM2.5':'σ (µgr/m3)'})

dfdata = pd.merge(dfcount, dfstd, on='created_at', how='inner')



# ambil 1 parameter

In [ ]:
# Gabungkan data per jam hanya dengan data yang tersedia 75%
df1hpm = dfpm
df1hpm = df1hpm.groupby(pd.Grouper(key='created_at',freq='1H')).mean().round(2)

df1hpm = pd.merge(dfdata, df1hpm, on='created_at', how='inner')

df1hpm['CV (%)'] = ((df1hpm['σ (µgr/m3)'] / df1hpm['PM2.5']) * 100).round(2)

df1hpm = df1hpm.loc[:,['created_at','(n)Data','σ (µgr/m3)','CV (%)','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari']]


df1hpm

,created_at,(n)Data,σ (µgr/m3),CV (%),Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari
0,2023-03-31 17:00:00+07:00,28,99.45,93.10,-288.96,-556.62,106.82,3.0,31.0
1,2023-03-31 18:00:00+07:00,30,65.48,127.97,-151.48,-250.20,51.17,3.0,31.0
2,2023-03-31 19:00:00+07:00,29,63.95,93.95,-100.15,-483.36,68.07,3.0,31.0
3,2023-03-31 20:00:00+07:00,30,53.67,94.94,-184.48,-459.17,56.53,3.0,31.0
4,2023-03-31 21:00:00+07:00,29,52.43,90.99,-155.45,-488.75,57.62,3.0,31.0
...,...,...,...,...,...,...,...,...,...
1672,2023-10-09 06:00:00+07:00,30,8.90,23.04,24.01,79.93,38.63,10.0,9.0
1673,2023-10-09 07:00:00+07:00,30,5.58,12.60,26.23,70.98,44.27,10.0,9.0
1674,2023-10-09 08:00:00+07:00,30,0.00,0.00,28.44,62.69,51.00,10.0,9.0
1675,2023-10-09 09:00:00+07:00,30,0.00,0.00,29.42,53.70,51.00,10.0,9.0


In [ ]:
df1hpm

,created_at,(n)Data,σ (µgr/m3),CV (%),Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari
0,2023-03-31 17:00:00+07:00,28,99.45,93.10,-288.96,-556.62,106.82,3.0,31.0
1,2023-03-31 18:00:00+07:00,30,65.48,127.97,-151.48,-250.20,51.17,3.0,31.0
2,2023-03-31 19:00:00+07:00,29,63.95,93.95,-100.15,-483.36,68.07,3.0,31.0
3,2023-03-31 20:00:00+07:00,30,53.67,94.94,-184.48,-459.17,56.53,3.0,31.0
4,2023-03-31 21:00:00+07:00,29,52.43,90.99,-155.45,-488.75,57.62,3.0,31.0
...,...,...,...,...,...,...,...,...,...
1672,2023-10-09 06:00:00+07:00,30,8.90,23.04,24.01,79.93,38.63,10.0,9.0
1673,2023-10-09 07:00:00+07:00,30,5.58,12.60,26.23,70.98,44.27,10.0,9.0
1674,2023-10-09 08:00:00+07:00,30,0.00,0.00,28.44,62.69,51.00,10.0,9.0
1675,2023-10-09 09:00:00+07:00,30,0.00,0.00,29.42,53.70,51.00,10.0,9.0


In [ ]:
path = '/content/gdrive/My Drive/SNI/TULT1/PM2.5/Hari/'


x=1
while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    dfhari = df1hpm[(df1hpm['Bulan']==x) & (df1hpm['Hari']==y) ]

    data = len(dfhari.axes[0])
    jumlah = str(data)
    if (dfhari.empty==True):
      pass
    else :
      std = dfhari['PM2.5'].std(numeric_only=True).round(2)
      mean = dfhari['PM2.5'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      dfhari.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
# 1H in Months
path = '/content/gdrive/My Drive/SNI/TULT1/PM2.5/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df1hpm[(df1hpm['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['PM2.5'].std(numeric_only=True).round(2)
    mean = dfbulan['PM2.5'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation 1H.csv", index=None)
  x+=1

In [ ]:
df8hpm = df1hpm
df8hpm = df8hpm.groupby(pd.Grouper(key='created_at',freq='8H')).mean().round(2).reset_index()
df8hpm = df8hpm.loc[:,['created_at','σ (µgr/m3)','CV (%)','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari']]



In [ ]:
# 8H in Months
path = '/content/gdrive/My Drive/SNI/TULT1/PM2.5/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df8hpm[(df8hpm['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['PM2.5'].std(numeric_only=True).round(2)
    mean = dfbulan['PM2.5'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation 8H.csv", index=None)
  x+=1